In [1]:
%matplotlib inline

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
#Load pre-trained embedding

In [3]:
import numpy as np
import pickle

In [37]:
words = []
idx = 0
word2idx = {}
vectors = []

with open('glove/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
#vectors = bcolz.carray(vectors[1:].reshape((400000, 50)), rootdir='../glove/6B.50.dat', mode='w')
#vectors.flush()
pickle.dump(words, open('glove/6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open('glove/6B.50_idx.pkl', 'wb'))

In [38]:
words = pickle.load(open('glove/6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open('glove/6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [39]:
glove['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [13]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            lineIds = eval(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [4]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)

NameError: name 'corpus' is not defined

In [5]:
# Read query/response pairs and return a voc object
def readVocs(corpus_name):
    print("Reading lines...")
    # Read the file and split into lines

    source1 = open('jokes-all-clean.txt', encoding='utf-8').\
        read().strip().split('\n')
    source1 = [normalizeString(s) for s in source1]
    
    
    # Split every line into pairs and normalize
    pairs = [s.split(". ", 1) for s in source1]
    
    pairs = [s  for s in pairs if len(s) == 2]
    
    return pairs

In [6]:
len(readVocs("story"))

Reading lines...


NameError: name 'normalizeString' is not defined

In [7]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [8]:
MAX_LENGTH = 300  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s.lower()


# Read query/response pairs and return a voc object
def readVocs(corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    print("Reading lines...")
    # Read the file and split into lines

    source1 = open('jokes-all-clean.txt', encoding='utf-8').\
        read().strip().split('\n')
    source1 = [normalizeString(s) for s in source1]
    
    
    # Split every line into pairs and normalize
    pairs = [s.split(". ", 1) for s in source1]
    
    pairs = [s  for s in pairs if len(s) == 2]
    
    
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus_name, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs



# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData("story", save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Reading lines...
Read 9144 sentence pairs
Trimmed to 9144 sentence pairs
Counting words...
Counted words: 14403

pairs:
['i m glad to be on cable ', 'the truth is i ve dreamed of being a talk show host on basic cable ever since i was .']
['it s not easy doing a late night show on a channel without a lot of money and that viewers have trouble finding ', 'so that s why i left nbc .']
['earlier today former president george w ', 'bush appeared on oprah winfrey . when asked about being the leader of the free world oprah winfrey said it s not bad .']
['apple just launched its online store in china ', 'apple says this is an exciting opportunity to sell ipods to the very kids who make them .']
['a police officer in london is in trouble for allegedly slipping references to songs into his official reports ', 'authorities became suspicious when they read the part that said time of death i like big butts and i cannot lie .']
['earlier this week a

In [9]:
MIN_COUNT = 1    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 14400 / 14400 = 1.0000
Trimmed from 9144 pairs to 9144, 1.0000 of total


In [10]:
voc.index2word.values()

dict_values(['PAD', 'SOS', 'EOS', '', 'chevrolet', 'doors', 'nutritious', 'nick', 'spinning', 'wonka', 'passengers', 'lengthen', 'hurting', 'coli', 'everest', 'differential', 'mosque', 'fries', 'portuguese', 'disappear', 'usually', 'whoopi', 'gotten', 'tsa', 'longevity', 'craigslist', 'burn', 'unbuttoning', 'arrest', 'brooklyn', 'rodrigo', 'visitors', 'bread', 'reads', 'hurricane', 'nice', 'chiefs', 'sesame', 'grandson', 'birth', 'councilman', 'philly', 'instance', 'resolution', 'ba', 'later', 'opportunity', 'marketing', 'defeating', 'someone', 'hostile', 'canerdians', 'curtain', 'pitcher', 'handlebar', 'hazed', 'racewalker', 'humanness', 'dancer', 'dustin', 'slam', 'media', 'argentinean', 'labeouf', 'its', 'antibiotics', 'ibiza', 'monitoring', 'ought', 'palin', 'shirtless', 'occupations', 'resist', 'caused', 'biathlon', 'matters', 'furious', 'cured', 'dignity', 'pakistan', 'rejected', 'zac', 'nfl', 'we', 'unreleased', 'prayers', 'unlocked', 'piglet', 'warhol', 'batcave', 'lithuania', 

In [12]:
len(weights_matrix[3])

50

In [15]:
weights_matrix.shape

(14403, 50)

In [17]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        #print(pair[0])
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[ 6271,   128,  9205,  5800, 12915],
        [ 6307,  5800,  5361,  7094,  2342],
        [ 8353,  6618,  1976, 11569,  6681],
        [ 2485,  1348, 12042, 11765,  9857],
        [11569,  2143,  6307,  5046,  2465],
        [ 6307, 12758,  3838,  4389,  8283],
        [12762,  4681, 11031, 10100,  7591],
        [ 7911,  3773, 14288,  5800,  6307],
        [12674,  5800, 13046,  4231, 11693],
        [ 6445,  5535,  2791,   726,     3],
        [11339,  9791, 12064,  9857,     2],
        [ 7591,  8336, 10643,  4002,     0],
        [ 5800, 10042,   714,  5458,     0],
        [ 2067,  8968, 13758,     3,     0],
        [ 1460, 10479,     3,     2,     0],
        [   92,     3,     2,     0,     0],
        [11388,     2,     0,     0,     0],
        [ 9791,     0,     0,     0,     0],
        [ 6307,     0,     0,     0,     0],
        [  545,     0,     0,     0,     0],
        [14206,     0,     0,     0,     0],
        [ 7591,     0,     0,     0,   

In [18]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [19]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [20]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [21]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [22]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [23]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f} Perplexity: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg, np.exp(print_loss_avg)) )
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [24]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
            #if decoder_input 
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [25]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            final_output = []
            for x in output_words:
                if not (x == 'EOS' or x == 'PAD'):
                    final_output.append(x)
                elif (x == 'EOS'):
                    break;
            output_sentence = ' '.join(final_output)
            print('Bot:', ' '.join(output_sentence))

        except KeyError:
            print("Error: Encountered unknown word.")

In [26]:
# Run Model
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    #print(num_embeddings)
    pre_trained = torch.FloatTensor(weights_matrix)
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    #print(emb_layer)
    emb_layer.load_state_dict({'weight': pre_trained})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [27]:
len(weights_matrix)

14403

In [28]:
voc.num_words

14403

In [29]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 300
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 16

# Set checkpoint to load from; set to None if starting from scratch
#loadFilename = None
checkpoint_iter = 2100
loadFilename = os.path.join(save_dir, model_name, "story",
                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
#print(weights_matrix[3])

#input = torch.LongTensor([3])
#print(emb_layer(input))

if loadFilename:
    embedding.load_state_dict(embedding_sd)
#else:
#    embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')


Building encoder and decoder ...
Models built and ready to go!


In [36]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 5000
print_every = 1
save_every = 300

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, "story", loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 2101; Percent complete: 42.0%; Average loss: 4.6139 Perplexity: 100.8755
Iteration: 2102; Percent complete: 42.0%; Average loss: 3.9206 Perplexity: 50.4310
Iteration: 2103; Percent complete: 42.1%; Average loss: 4.5169 Perplexity: 91.5535
Iteration: 2104; Percent complete: 42.1%; Average loss: 4.2467 Perplexity: 69.8737
Iteration: 2105; Percent complete: 42.1%; Average loss: 4.0924 Perplexity: 59.8813
Iteration: 2106; Percent complete: 42.1%; Average loss: 4.1306 Perplexity: 62.2142
Iteration: 2107; Percent complete: 42.1%; Average loss: 4.2827 Perplexity: 72.4382
Iteration: 2108; Percent complete: 42.2%; Average loss: 4.4472 Perplexity: 85.3886
Iteration: 2109; Percent complete: 42.2%; Average loss: 4.3505 Perplexity: 77.5166
Iteration: 2110; Percent complete: 42.2%; Average loss: 4.0283 Perplexity: 56.1673
Iteration: 2111; Percent complete: 42.2%; Average loss: 4.0538 Perplexity: 57.6143
Iteration: 211

Iteration: 2200; Percent complete: 44.0%; Average loss: 4.2335 Perplexity: 68.9551
Iteration: 2201; Percent complete: 44.0%; Average loss: 4.3623 Perplexity: 78.4389
Iteration: 2202; Percent complete: 44.0%; Average loss: 4.3868 Perplexity: 80.3849
Iteration: 2203; Percent complete: 44.1%; Average loss: 4.3859 Perplexity: 80.3081
Iteration: 2204; Percent complete: 44.1%; Average loss: 4.3219 Perplexity: 75.3298
Iteration: 2205; Percent complete: 44.1%; Average loss: 4.3382 Perplexity: 76.5706
Iteration: 2206; Percent complete: 44.1%; Average loss: 3.8638 Perplexity: 47.6479
Iteration: 2207; Percent complete: 44.1%; Average loss: 4.2834 Perplexity: 72.4895
Iteration: 2208; Percent complete: 44.2%; Average loss: 3.9995 Perplexity: 54.5699
Iteration: 2209; Percent complete: 44.2%; Average loss: 4.7305 Perplexity: 113.3566
Iteration: 2210; Percent complete: 44.2%; Average loss: 4.1685 Perplexity: 64.6192
Iteration: 2211; Percent complete: 44.2%; Average loss: 4.2941 Perplexity: 73.2636
Ite

Iteration: 2300; Percent complete: 46.0%; Average loss: 4.3931 Perplexity: 80.8946
Iteration: 2301; Percent complete: 46.0%; Average loss: 4.5816 Perplexity: 97.6729
Iteration: 2302; Percent complete: 46.0%; Average loss: 3.8930 Perplexity: 49.0593
Iteration: 2303; Percent complete: 46.1%; Average loss: 4.0570 Perplexity: 57.7984
Iteration: 2304; Percent complete: 46.1%; Average loss: 4.3138 Perplexity: 74.7212
Iteration: 2305; Percent complete: 46.1%; Average loss: 4.1170 Perplexity: 61.3759
Iteration: 2306; Percent complete: 46.1%; Average loss: 3.6816 Perplexity: 39.7097
Iteration: 2307; Percent complete: 46.1%; Average loss: 4.4826 Perplexity: 88.4641
Iteration: 2308; Percent complete: 46.2%; Average loss: 3.8087 Perplexity: 45.0937
Iteration: 2309; Percent complete: 46.2%; Average loss: 4.1330 Perplexity: 62.3658
Iteration: 2310; Percent complete: 46.2%; Average loss: 3.8442 Perplexity: 46.7198
Iteration: 2311; Percent complete: 46.2%; Average loss: 4.3496 Perplexity: 77.4455
Iter

Iteration: 2400; Percent complete: 48.0%; Average loss: 4.2904 Perplexity: 72.9930
Iteration: 2401; Percent complete: 48.0%; Average loss: 4.2855 Perplexity: 72.6386
Iteration: 2402; Percent complete: 48.0%; Average loss: 4.1375 Perplexity: 62.6483
Iteration: 2403; Percent complete: 48.1%; Average loss: 3.8070 Perplexity: 45.0162
Iteration: 2404; Percent complete: 48.1%; Average loss: 4.5511 Perplexity: 94.7400
Iteration: 2405; Percent complete: 48.1%; Average loss: 4.2540 Perplexity: 70.3891
Iteration: 2406; Percent complete: 48.1%; Average loss: 4.2899 Perplexity: 72.9604
Iteration: 2407; Percent complete: 48.1%; Average loss: 4.2413 Perplexity: 69.4951
Iteration: 2408; Percent complete: 48.2%; Average loss: 4.0610 Perplexity: 58.0348
Iteration: 2409; Percent complete: 48.2%; Average loss: 4.1283 Perplexity: 62.0746
Iteration: 2410; Percent complete: 48.2%; Average loss: 4.0058 Perplexity: 54.9164
Iteration: 2411; Percent complete: 48.2%; Average loss: 3.8142 Perplexity: 45.3423
Iter

Iteration: 2500; Percent complete: 50.0%; Average loss: 3.9143 Perplexity: 50.1149
Iteration: 2501; Percent complete: 50.0%; Average loss: 4.1739 Perplexity: 64.9670
Iteration: 2502; Percent complete: 50.0%; Average loss: 3.8470 Perplexity: 46.8535
Iteration: 2503; Percent complete: 50.1%; Average loss: 4.6012 Perplexity: 99.6012
Iteration: 2504; Percent complete: 50.1%; Average loss: 3.6830 Perplexity: 39.7664
Iteration: 2505; Percent complete: 50.1%; Average loss: 4.5936 Perplexity: 98.8514
Iteration: 2506; Percent complete: 50.1%; Average loss: 3.8748 Perplexity: 48.1713
Iteration: 2507; Percent complete: 50.1%; Average loss: 4.1788 Perplexity: 65.2884
Iteration: 2508; Percent complete: 50.2%; Average loss: 3.4364 Perplexity: 31.0749
Iteration: 2509; Percent complete: 50.2%; Average loss: 4.1257 Perplexity: 61.9085
Iteration: 2510; Percent complete: 50.2%; Average loss: 4.2109 Perplexity: 67.4158
Iteration: 2511; Percent complete: 50.2%; Average loss: 4.1982 Perplexity: 66.5640
Iter

...

Iteration: 4586; Percent complete: 91.7%; Average loss: 3.6187 Perplexity: 37.2879
Iteration: 4587; Percent complete: 91.7%; Average loss: 3.3785 Perplexity: 29.3267
Iteration: 4588; Percent complete: 91.8%; Average loss: 3.5904 Perplexity: 36.2472
Iteration: 4589; Percent complete: 91.8%; Average loss: 3.3258 Perplexity: 27.8205
Iteration: 4590; Percent complete: 91.8%; Average loss: 3.1380 Perplexity: 23.0575
Iteration: 4591; Percent complete: 91.8%; Average loss: 3.4553 Perplexity: 31.6686
Iteration: 4592; Percent complete: 91.8%; Average loss: 3.1608 Perplexity: 23.5903
Iteration: 4593; Percent complete: 91.9%; Average loss: 2.9965 Perplexity: 20.0153
Iteration: 4594; Percent complete: 91.9%; Average loss: 3.4194 Perplexity: 30.5521
Iteration: 4595; Percent complete: 91.9%; Average loss: 3.2025 Perplexity: 24.5938
Iteration: 4596; Percent complete: 91.9%; Average loss: 3.5928 Perplexity: 36.3363
Iteration: 4597; Percent complete: 91.9%; Average loss: 3.3618 Perplexity: 28.8413
Iter

Iteration: 4685; Percent complete: 93.7%; Average loss: 3.0167 Perplexity: 20.4228
Iteration: 4686; Percent complete: 93.7%; Average loss: 3.4797 Perplexity: 32.4493
Iteration: 4687; Percent complete: 93.7%; Average loss: 3.5265 Perplexity: 34.0044
Iteration: 4688; Percent complete: 93.8%; Average loss: 3.2209 Perplexity: 25.0510
Iteration: 4689; Percent complete: 93.8%; Average loss: 3.1831 Perplexity: 24.1218
Iteration: 4690; Percent complete: 93.8%; Average loss: 3.8233 Perplexity: 45.7547
Iteration: 4691; Percent complete: 93.8%; Average loss: 3.3507 Perplexity: 28.5228
Iteration: 4692; Percent complete: 93.8%; Average loss: 3.2122 Perplexity: 24.8335
Iteration: 4693; Percent complete: 93.9%; Average loss: 3.3279 Perplexity: 27.8808
Iteration: 4694; Percent complete: 93.9%; Average loss: 3.6843 Perplexity: 39.8183
Iteration: 4695; Percent complete: 93.9%; Average loss: 3.3032 Perplexity: 27.1983
Iteration: 4696; Percent complete: 93.9%; Average loss: 2.8420 Perplexity: 17.1504
Iter

Iteration: 4784; Percent complete: 95.7%; Average loss: 3.3213 Perplexity: 27.6967
Iteration: 4785; Percent complete: 95.7%; Average loss: 3.4752 Perplexity: 32.3045
Iteration: 4786; Percent complete: 95.7%; Average loss: 3.1523 Perplexity: 23.3893
Iteration: 4787; Percent complete: 95.7%; Average loss: 3.2550 Perplexity: 25.9209
Iteration: 4788; Percent complete: 95.8%; Average loss: 3.4243 Perplexity: 30.7020
Iteration: 4789; Percent complete: 95.8%; Average loss: 3.0849 Perplexity: 21.8646
Iteration: 4790; Percent complete: 95.8%; Average loss: 3.2059 Perplexity: 24.6774
Iteration: 4791; Percent complete: 95.8%; Average loss: 3.4293 Perplexity: 30.8543
Iteration: 4792; Percent complete: 95.8%; Average loss: 3.0073 Perplexity: 20.2318
Iteration: 4793; Percent complete: 95.9%; Average loss: 2.7229 Perplexity: 15.2242
Iteration: 4794; Percent complete: 95.9%; Average loss: 3.1676 Perplexity: 23.7513
Iteration: 4795; Percent complete: 95.9%; Average loss: 3.0071 Perplexity: 20.2293
Iter

Iteration: 4884; Percent complete: 97.7%; Average loss: 3.3416 Perplexity: 28.2648
Iteration: 4885; Percent complete: 97.7%; Average loss: 3.5199 Perplexity: 33.7817
Iteration: 4886; Percent complete: 97.7%; Average loss: 3.3774 Perplexity: 29.2936
Iteration: 4887; Percent complete: 97.7%; Average loss: 3.3489 Perplexity: 28.4722
Iteration: 4888; Percent complete: 97.8%; Average loss: 3.0200 Perplexity: 20.4911
Iteration: 4889; Percent complete: 97.8%; Average loss: 3.0296 Perplexity: 20.6899
Iteration: 4890; Percent complete: 97.8%; Average loss: 2.8783 Perplexity: 17.7838
Iteration: 4891; Percent complete: 97.8%; Average loss: 3.0978 Perplexity: 22.1501
Iteration: 4892; Percent complete: 97.8%; Average loss: 2.9748 Perplexity: 19.5851
Iteration: 4893; Percent complete: 97.9%; Average loss: 3.5143 Perplexity: 33.5910
Iteration: 4894; Percent complete: 97.9%; Average loss: 3.4072 Perplexity: 30.1793
Iteration: 4895; Percent complete: 97.9%; Average loss: 3.6548 Perplexity: 38.6602
Iter

Iteration: 4983; Percent complete: 99.7%; Average loss: 3.1876 Perplexity: 24.2299
Iteration: 4984; Percent complete: 99.7%; Average loss: 3.0721 Perplexity: 21.5882
Iteration: 4985; Percent complete: 99.7%; Average loss: 2.9787 Perplexity: 19.6621
Iteration: 4986; Percent complete: 99.7%; Average loss: 3.0946 Perplexity: 22.0793
Iteration: 4987; Percent complete: 99.7%; Average loss: 2.8371 Perplexity: 17.0663
Iteration: 4988; Percent complete: 99.8%; Average loss: 3.4621 Perplexity: 31.8835
Iteration: 4989; Percent complete: 99.8%; Average loss: 2.9614 Perplexity: 19.3248
Iteration: 4990; Percent complete: 99.8%; Average loss: 3.3500 Perplexity: 28.5038
Iteration: 4991; Percent complete: 99.8%; Average loss: 3.0944 Perplexity: 22.0732
Iteration: 4992; Percent complete: 99.8%; Average loss: 3.2218 Perplexity: 25.0723
Iteration: 4993; Percent complete: 99.9%; Average loss: 2.9366 Perplexity: 18.8523
Iteration: 4994; Percent complete: 99.9%; Average loss: 3.3008 Perplexity: 27.1346
Iter

In [37]:
torch.save({
            'encoder': encoder.state_dict(),
            'decoder': decoder.state_dict(),
            'en_opt': encoder_optimizer.state_dict(),
            'de_opt': decoder_optimizer.state_dict(),
            }, 'model')

In [30]:
checkpoint = torch.load('model')
encoder.load_state_dict(checkpoint['encoder'])
decoder.load_state_dict(checkpoint['decoder'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [ ]:
encoder.load_state_dict(encoder_sd)
LuongAttnDecoderRNN.load_state_dict(decoder_sd)

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> a city street with a man in the background
Bot: t h e   m a n   i s   s o   b a d   h e   s   g o i n g   t o   b e   t h e   o n e   t o   g e t   t h e   n e w   m o m e n t   .
> a man is cutting a pizza with a knife
Bot: h e   s   g o i n g   t o   b e   t h e   o n e   t o   t e l l   h e r   h e   s   s t i l l   s t i l l   i n   t h e   h o s p i t a l   .
> a close up of a plate of broccoli and meat
Error: Encountered unknown word.
>  a close up of a plate of broccoli and meat
Error: Encountered unknown word.
> a bear is walking through the water near a forest
Bot: i n   o t h e r   w o r d s   t h e   p o p e   i s   s t i l l   i n   t h e   m i d d l e   o f   t h e   f i e l d   .
> a skier is coming down the slope on a sunny day
Error: Encountered unknown word.
> a woman walking down a street holding an umbrella
Bot: t h e   w o m a n   i s   s o   b a d   h e   w a s   a b l e   t o   b e   t o u c h e d   t h e   p h r a s e   h e y   i   m   i n v e s t i g a t i n  

In [31]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words

In [32]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, searcher,voc, pair[0])
        # Format and print response sentence
        #output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
        final_output = []
        for x in output_words:
            if not (x == 'EOS' or x == 'PAD'):
                final_output.append(x)
            elif (x == 'EOS'):
                break;
        output_sentence = ' '.join(final_output)
        print('<', output_sentence)
        print('')

In [35]:
evaluateRandomly(encoder, decoder)

> oprah winfrey is now th on fortune s list of most powerful women in business 
= in a related story oprah winfrey just released o magazine s list of women to destroy .
< in a related story the report said you know what ? i m going to do to be a job ?

> michigan has put forth a plan to give visas to immigrants if they promise to spend five years living in detroit 
= when they heard immigrants from dozens of impoverished war torn countries said we re good .
< the ceo said they re going to be the one to tell it to the people who make them .

> kobe bryant has a lot of work to do to get ready for the season 
= we just played one on one and he only beat me .
< the first thing that s the first time you re the first time in the show that you re the same season .

> in florida justin bieber was arrested for dui 
= police reports said bieber s blood contained large amounts of alcohol pot and flinstones chewables .
< he said he can t wait to rent his name to his th view of his car .

> today t

In [56]:
output_words = evaluate(encoder, decoder, searcher,voc, "play station")

In [57]:
output_sentence = ' '.join(output_words)
print('<', output_sentence)

< I 'm not a good man . I 'm not a good thing . <newline> <newline> I 'm not a good , but I 'm not a good . <newline> <newline> I 'm not a good , but I 'm not a good . <newline> <newline> I 'm not a good , but I 'm not a good . <newline> <newline> I 'm not a good , but I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS , I 'm not a good . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS . EOS .
